In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [9]:
torch.manual_seed(42)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
file_location_train = "/content/drive/MyDrive/PyTorch/Dataset/fashion-mnist_train.csv"
file_location_test = "/content/drive/MyDrive/PyTorch/Dataset/fashion-mnist_test.csv"

In [24]:
train_data = pd.read_csv(file_location_train)
train_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
test_data = pd.read_csv(file_location_test)
test_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
X_train = train_data.iloc[: , 1 : ].values
y_train = train_data.iloc[:, 0].values
X_test = test_data.iloc[ : , 1 : ].values
y_test = test_data.iloc[ : , 0].values

In [28]:
print(f"Shape of X_train, y_train, X_test, y_test: {X_train.shape, y_train.shape, X_test.shape, y_test.shape}")

Shape of X_train, y_train, X_test, y_test: ((60000, 784), (60000,), (10000, 784), (10000,))


In [29]:
X_train = X_train/255.0
X_test = X_test/255.0

In [30]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype = torch.long)

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [31]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [32]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True, pin_memory = True)

In [33]:
len(train_loader)

1875

In [35]:
class MyNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )

  def forward(self, x):
    return self.model(x)

In [36]:
learning_rate = 0.1
epochs = 100

## 1. Move the model to GPU
- Move your model to the selected device (cuda for GPU or cpu) so that all the computations occur on the same device.

In [37]:
# Instatiate the model
model = MyNN(X_train.shape[1])

# Model to GPU
model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

## 2. Modify the Training loop by moving data to GPU.
- Ensure that each batch of data (features and labels) is moved to the GPU before processing. This ensures that both the model and data are on the same device.

In [38]:
# training loop
for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # Move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # Forward pass
    outputs = model(batch_features)

    # Calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

Epoch: 1 , Loss: 0.6010189413030942
Epoch: 2 , Loss: 0.4135076250910759
Epoch: 3 , Loss: 0.37121313376426696
Epoch: 4 , Loss: 0.3437591538945834
Epoch: 5 , Loss: 0.32385402583877243
Epoch: 6 , Loss: 0.3091171885728836
Epoch: 7 , Loss: 0.2962389718155066
Epoch: 8 , Loss: 0.28478200549980004
Epoch: 9 , Loss: 0.27343885239958765
Epoch: 10 , Loss: 0.26577090840637685
Epoch: 11 , Loss: 0.25830729249715806
Epoch: 12 , Loss: 0.25099953015943366
Epoch: 13 , Loss: 0.24375854245920975
Epoch: 14 , Loss: 0.23399951468805472
Epoch: 15 , Loss: 0.23218018555541833
Epoch: 16 , Loss: 0.22519357588986555
Epoch: 17 , Loss: 0.22086197706957658
Epoch: 18 , Loss: 0.21513471797307332
Epoch: 19 , Loss: 0.21071677063380678
Epoch: 20 , Loss: 0.205938256465892
Epoch: 21 , Loss: 0.20132937553326288
Epoch: 22 , Loss: 0.19747952504952748
Epoch: 23 , Loss: 0.1947750591923793
Epoch: 24 , Loss: 0.18993149445156257
Epoch: 25 , Loss: 0.18798903200775385
Epoch: 26 , Loss: 0.18390918584913016
Epoch: 27 , Loss: 0.181061388

In [39]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

## 3. Modify the Evaluation loop by moving data to GPU.
- Similarly, ensure test data is moved to the GPU during evaluation. Disable gradient calculations using torch.no_grad() for efficiency.  

In [40]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8811


## 4. Optimize the GPU Usage.

To make the best use of GPU resources, apply the following optimizations:

**a. Use Larger Batch Sizes**
- Larger batch sizes can better utilize GPU memory and reduce computation time per epoch (if memory allows).  
- batch_size = 128

**b. Enable DataLoader Pinning**
- Use pin_memory = True in DataLoader to speed up data transfer from CPU to GPU.
- pin_memory = True

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 128, shuffle = True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size = 128, shuffle = True, pin_memory = True)